In [1]:
# 1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.12，升级机器人电机驱动固件版本到0.2.10.11，详细使用请参考上位机使用说明
# 2. 请使用上位机对机器人零位进行标定（无绝编则不用）
# 3. 请使用上位机调整机器人电机pd参数（注意不是pid参数），参考值：上肢（头腰双臂）p：200，d：10，可以根据软硬自己调节,腕关节可以小一点
# 4. 请使用指定的docker镜像进行开发，镜像名为fourier_hardware:v2.3，请使用开源仓库中的docker启动脚本启动docker

In [1]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# grx_sot_py 用于对机器人进行运动学，动力学求解
import grx_sot_py

In [2]:
# 加载对应机型的config配置文件
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr1t2")

In [3]:
# 配置各控制组的控制模式为PD模式，并忽略下肢控制的运动
config.changeControlGroupMotorMode("left_manipulator","pd")
config.changeControlGroupMotorMode("right_manipulator","pd")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_wrist","pd")
config.changeControlGroupMotorMode("right_wrist","pd")

True

In [4]:
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr1t2",config)

In [5]:
# 根据配置文件定义对应的机器人类，支持运动学、动力学求解
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr1t2", grx_sot_py.Flags.ignore_collisions)

In [6]:
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joint_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joint_vel_order[group_name] = joint_vel_orders

In [7]:
print(config.getRobotName())
print(config.getControlGroupNames())

gr1t2
['waist', 'left_manipulator', 'right_manipulator', 'left_wrist', 'right_wrist']


In [8]:
# 使能整个机器人
hardware.enableRobot()

MOTOR: 192.168.137.10, ENABLE SUCCESS! 
MOTOR: 192.168.137.11, ENABLE SUCCESS! 
MOTOR: 192.168.137.12, ENABLE SUCCESS! 
MOTOR: 192.168.137.13, ENABLE SUCCESS! 
MOTOR: 192.168.137.14, ENABLE SUCCESS! 
MOTOR: 192.168.137.15, ENABLE SUCCESS! 
MOTOR: 192.168.137.16, ENABLE SUCCESS! 
MOTOR: 192.168.137.30, ENABLE SUCCESS! 
MOTOR: 192.168.137.31, ENABLE SUCCESS! 
MOTOR: 192.168.137.32, ENABLE SUCCESS! 
MOTOR: 192.168.137.33, ENABLE SUCCESS! 
MOTOR: 192.168.137.34, ENABLE SUCCESS! 
MOTOR: 192.168.137.35, ENABLE SUCCESS! 
MOTOR: 192.168.137.36, ENABLE SUCCESS! 
MOTOR: 192.168.137.90, ENABLE SUCCESS! 
MOTOR: 192.168.137.91, ENABLE SUCCESS! 
MOTOR: 192.168.137.92, ENABLE SUCCESS! 


True

In [26]:
# 获取机器人当前状态下的对应控制组['waist', 'head', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg']的关节qpos(rad),
lm = hardware.getControlGroupState("left_manipulator")
rm = hardware.getControlGroupState("right_manipulator")
print("left_manipulator:", lm.q)
print("right_manipulator", rm.q)

left_manipulator: [ 0.          0.          0.          0.          0.          0.
  0.         -1.36672574  1.36937721 -1.36837274 -1.37124306 -1.36364223]
right_manipulator [ 0.          0.          0.          0.          0.          0.
  0.         -1.36205033 -1.36524138  1.37764295 -1.36844444  1.36419839]


In [27]:
# 生成机器人双臂关节从当前位置到零位的轨迹
import numpy as np
point_init = [0.0, 0.0, 0.0, 0.0, 0.0]
rm_traj = fourier_hardware_py.bridgeTrajectory(rm.q[-5:],point_init)
lm_traj = fourier_hardware_py.bridgeTrajectory(lm.q[-5:],point_init)

In [28]:
# 执行回零轨迹，驱动右臂控制组关节回到零位
import time

for i in range(len(rm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    hardware.setControlGroupPosCmd("right_manipulator",rm_traj[1][i],np.zeros(5),gravity_torque[group_joint_vel_order["right_manipulator"]])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

control loop is slow 0.006937863305211067
control loop is slow 0.0027541150338947773
control loop is slow 0.012103808112442493
control loop is slow 0.013235929887741804
control loop is slow 0.01357349380850792
control loop is slow 0.004902403801679611
control loop is slow 0.007239543832838535
control loop is slow 0.008516808971762657
control loop is slow 0.003897939808666706
control loop is slow 0.007938074879348278
control loop is slow 0.003929995931684971
control loop is slow 0.003923380747437477
control loop is slow 0.004141265992075205
control loop is slow 0.003773747943341732
control loop is slow 0.0039005777798593044
control loop is slow 0.003993005957454443
control loop is slow 0.006888950243592262
control loop is slow 0.003512697760015726
control loop is slow 0.00936691602692008
control loop is slow 0.009295476134866476
control loop is slow 0.010106150060892105
control loop is slow 0.004476495087146759
control loop is slow 0.003217726945877075
control loop is slow 0.00294817890

In [29]:
# 驱动左臂控制组关节回到零位
import time

for i in range(len(lm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    hardware.setControlGroupPosCmd("left_manipulator",lm_traj[1][i],np.zeros(5),gravity_torque[group_joint_vel_order["left_manipulator"]])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

control loop is slow 0.005883842706680298
control loop is slow 0.007488282863050699
control loop is slow 0.006351142190396786
control loop is slow 0.003498949110507965
control loop is slow 0.00839491793885827
control loop is slow 0.008197998162359
control loop is slow 0.009942487813532352
control loop is slow 0.022116306237876415
control loop is slow 0.005294763948768377
control loop is slow 0.0031844759359955788
control loop is slow 0.0030730371363461018
control loop is slow 0.0026962379924952984
control loop is slow 0.013121087104082108
control loop is slow 0.00838461983948946
control loop is slow 0.006165539380162954
control loop is slow 0.003178847022354603
control loop is slow 0.013032183051109314
control loop is slow 0.0026063742116093636
control loop is slow 0.0034116990864276886
control loop is slow 0.0057834237813949585
control loop is slow 0.013054232578724623
control loop is slow 0.006928282789885998
control loop is slow 0.003670917823910713
control loop is slow 0.0041161170

In [23]:
# 从模型计算全身关节对重力进行补偿的力矩值
regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
params = robot.get_inertia_params()
gravity_torque = regressor.dot(params)
print(gravity_torque)
print(gravity_torque[group_joint_vel_order["left_manipulator"]])

[-2.39301207e-01 -2.59756241e-02  5.34971617e+02 -1.47859419e-01
  1.28312520e+00  6.08229962e-02  8.66767318e-01  0.00000000e+00
 -5.24430534e-01 -4.10816509e-01 -3.85228890e-01  1.04967000e-03
 -8.75278768e-01  0.00000000e+00 -5.24417683e-01 -4.10885572e-01
 -3.85228890e-01 -1.04967000e-03  6.08229962e-02 -5.38505973e-01
  1.17020624e-01 -2.79302251e-02 -4.20124498e-01 -6.85663726e-04
 -1.50905087e-02  9.31357972e-01  6.30290200e-04  3.66287141e-04
  8.42403515e-05  3.10017036e-03  3.57596057e-04 -7.94352162e-02
 -9.47238915e-01  1.46049534e-04 -1.79564080e-02  1.91249030e-04
 -2.48025628e-03  2.76784415e-04]
[-1.50905087e-02  9.31357972e-01  6.30290200e-04  3.66287141e-04
  8.42403515e-05]


In [25]:
import numpy as np
import math
position_desired = np.zeros(17)  # deg
ERROR_CODE = 50 #关节组报错的错误码

# 设置循环频率的时间
sim_dt = 0.0025
sim_duration = 60.00
sim_steps = int(sim_duration / sim_dt)

for k in range(sim_steps):
    start_time = time.perf_counter()
    # current control
    # position_desired[0] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # waist_yaw_joint

    # position_desired[1] = -(15 * math.sin(0.0025 * k)) # head_yaw_joint
    # position_desired[2] = -(15 * math.sin(0.0025 * k)) # head_pitch_joint
    
    position_desired[3] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_pitch_joint
    position_desired[4] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # left_shoulder_roll_joint
    position_desired[5] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_yaw_joint
    position_desired[6] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_elbow_pitch_joint
    position_desired[7] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_wrist_yaw_joint
    position_desired[8] = -(10 * math.sin(0.0025 * k)) # left_wrist_roll_joint
    position_desired[9] = -(10 * math.sin(0.0025 * k)) # left_wrist_pitch_joint
    
    position_desired[10] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_pitch_joint
    position_desired[11] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_roll_joint
    position_desired[12] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_shoulder_yaw_joint
    position_desired[13] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_elbow_pitch_joint
    position_desired[14] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_wrist_yaw_joint
    position_desired[15] = -(10 * math.sin(0.0025 * k)) # right_wrist_roll_joint
    position_desired[16] =  -(10 * math.sin(0.0025 * k)) # right_wrist_pitch_joint

    #将控制命令从关节角度转换为弧度
    position_desired = np.deg2rad(position_desired)

    # 返回一个字典，字典的key是control group 的名字，value是连续失去连接的次数（最大观测值是50），如果value等于50，代表该control group 失去连接
    # 应该在程序控制周期内实时观测次值，保证程序运行的安全
    motor_stats = hardware.getGroupsLoss()
    #检查机器人真机的硬件状态，如果机器人使能后某个电机断连，将会切断控制循环命令，并答应错误信息。
    if ERROR_CODE in motor_stats.values():
        print(f"motor get error.")
        break

    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    
    # 针对双臂控制组进行控制，需要注意对应的输入：关节控制组、对应维度的关节位置信息、速度信息(设置为0)、前馈的力矩值
    hardware.setControlGroupPosCmd("left_manipulator",position_desired[3:8],np.zeros(5),gravity_torque[group_joint_vel_order["left_manipulator"]])
    hardware.setControlGroupPosCmd("right_manipulator",position_desired[10:15],np.zeros(5),gravity_torque[group_joint_vel_order["right_manipulator"]])
    # hardware.setControlGroupPosCmd("left_manipulator",position_desired[3:8])
    # hardware.setControlGroupPosCmd("right_manipulator",position_desired[10:15])
    hardware.setControlGroupPosCmd("left_wrist",position_desired[8:10])
    hardware.setControlGroupPosCmd("right_wrist",position_desired[15:17])
    
    # hardware.setControlGroupPosCmd("waist", np.array([position_desired[0]]))
    # hardware.setControlGroupPosCmd("head", position_desired[1:3])
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)


control loop is slow 0.008260991889983416
control loop is slow 0.004629326984286308
control loop is slow 0.004059305880218744
control loop is slow 0.0027625998482108116
control loop is slow 0.002662623766809702
control loop is slow 0.009646208956837654
control loop is slow 0.011339984834194183
control loop is slow 0.004443767014890909
control loop is slow 0.005429214332252741
control loop is slow 0.007260856684297323
control loop is slow 0.003066388890147209
control loop is slow 0.004199496936053038
control loop is slow 0.009832579176872969
control loop is slow 0.010852804873138666
control loop is slow 0.02292163297533989
control loop is slow 0.004826165735721588
control loop is slow 0.009853540919721127
control loop is slow 0.01082554180175066
control loop is slow 0.006278468761593103
control loop is slow 0.003433952108025551
control loop is slow 0.0033212550915777683
control loop is slow 0.011129696853458881
control loop is slow 0.005444778129458427
control loop is slow 0.01231788797

In [14]:
hardware.disableRobot()

MOTOR: 192.168.137.93, DISABLE SUCCESS! 


True

MOTOR: 192.168.137.95, DISABLE SUCCESS! 
MOTOR: 192.168.137.10, DISABLE SUCCESS! 
MOTOR: 192.168.137.11, DISABLE SUCCESS! 
MOTOR: 192.168.137.12, DISABLE SUCCESS! 
MOTOR: 192.168.137.13, DISABLE SUCCESS! 
MOTOR: 192.168.137.14, DISABLE SUCCESS! 
MOTOR: 192.168.137.15, DISABLE SUCCESS! 
MOTOR: 192.168.137.16, DISABLE SUCCESS! 
MOTOR: 192.168.137.30, DISABLE SUCCESS! 
MOTOR: 192.168.137.31, DISABLE SUCCESS! 
MOTOR: 192.168.137.32, DISABLE SUCCESS! 
MOTOR: 192.168.137.33, DISABLE SUCCESS! 
MOTOR: 192.168.137.34, DISABLE SUCCESS! 
MOTOR: 192.168.137.35, DISABLE SUCCESS! 
MOTOR: 192.168.137.36, DISABLE SUCCESS! 
MOTOR: 192.168.137.90, DISABLE SUCCESS! 
[2025-02-28 09:55:32] [12182] [error] head loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] right_manipulator loss message more than 50 times .
[2025-02-28 09:55:33] [12182] [error] head loss message more than 50 times .
[2025-02-28